In [117]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
from os.path import abspath, curdir, join, exists
import gffutils
import re

In [120]:
data_dir = abspath(join(curdir, "..", "data"))
public_dir = abspath(join(curdir, "..", "frontend", "public"))

GCF_000002765.6.gff from the assembly of Plasmodium falciparum 3D7: https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000002765.6/

In [139]:
recreate = True
if not exists(join(public_dir, "GCF_000002765.6.db")) or recreate:
    gff_db = gffutils.create_db(
        join(data_dir, "GCF_000002765.6", "genomic.gff"),
        join(public_dir, "GCF_000002765.6.db"),
        merge_strategy="merge",
        force=True,
    )
else:
    gff_db = gffutils.FeatureDB(join(public_dir, "GCF_000002765.6.db"))

In [140]:
# gff_db = sqlite3.connect(join(public_dir, "GCF_000002765.6.db"))
gff_db.conn.execute(
    "create table sequences (seqid text not null, start int, seq text not null, primary key (seqid, start))"
)

In [142]:
fna = join(data_dir, "GCF_000002765.6", "GCF_000002765.6_GCA_000002765_genomic.fna")
seqid = None
start = 0
with open(fna) as f:
    for line in f:
        if line.startswith(">"):
            seqid = re.search(r">([^\s]+)\s", line).group(1)
            start = 0
            print(f"Seq ID: {seqid}")
            continue
        if seqid is None:
            raise Exception("No seqid found")
        seq = line.strip()
        gff_db.conn.execute("insert into sequences values (?, ?, ?)", (seqid, start, seq))
        gff_db.conn.commit()
        start += len(seq)

Seq ID: NC_004325.2
Seq ID: NC_037280.1
Seq ID: NC_000521.4
Seq ID: NC_004318.2
Seq ID: NC_004326.2
Seq ID: NC_004327.3
Seq ID: NC_004328.3
Seq ID: NC_004329.3
Seq ID: NC_004330.2
Seq ID: NC_037281.1
Seq ID: NC_037282.1
Seq ID: NC_037284.1
Seq ID: NC_004331.3
Seq ID: NC_037283.1
